In [76]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from google.colab import drive
import pickle as pkl
import joblib
drive.mount('/content/drive')
#import streamlit as st

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
#loading the datasets
legacy = pd.read_csv('legacy.csv',na_values = "")
play_22 = pd.read_csv('players_22.csv')

legacy
play_22

<ipython-input-77-ef64be305e5d>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy = pd.read_csv('legacy.csv',na_values = "")
<ipython-input-77-ef64be305e5d>:3: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  play_22 = pd.read_csv('players_22.csv')


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [78]:
#picking relevant reafutures
corrcolumns = ['overall', 'potential', 'age', 'height_cm', 'weight_kg', 'pace', 'shooting', 'passing',
                     'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
                     'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys',
                     'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
                     'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                     'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
                     'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions',
                     'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure',
                     'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle',
                     'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning',
                     'goalkeeping_reflexes', 'goalkeeping_speed']

# Calculate correlation with 'overall' and sort
corrs= legacy[corrcolumns].corr()

highcorrs = corrs['overall'].sort_values(ascending=False)
highcorrs


overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
mentality_composure            0.683307
passing                        0.681114
dribbling                      0.606575
physic                         0.513279
attacking_short_passing        0.499757
mentality_vision               0.493282
skill_long_passing             0.483808
shooting                       0.482498
power_shot_power               0.480297
age                            0.458390
skill_ball_control             0.457011
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
goalkeeping_speed              0.385093
attacking_volleys              0.377048
skill_dribbling                0.369827
power_stamina                  0.360025
power_strength                 0.350786
mentality_positioning          0.350178


In [79]:
# selecting top 30 features
corrcolumns = highcorrs.index[1:50].tolist()
corrcolumns.append('overall')
corrcolumns

['movement_reactions',
 'potential',
 'mentality_composure',
 'passing',
 'dribbling',
 'physic',
 'attacking_short_passing',
 'mentality_vision',
 'skill_long_passing',
 'shooting',
 'power_shot_power',
 'age',
 'skill_ball_control',
 'skill_curve',
 'power_long_shots',
 'mentality_aggression',
 'attacking_crossing',
 'skill_fk_accuracy',
 'goalkeeping_speed',
 'attacking_volleys',
 'skill_dribbling',
 'power_stamina',
 'power_strength',
 'mentality_positioning',
 'attacking_heading_accuracy',
 'mentality_penalties',
 'attacking_finishing',
 'defending',
 'mentality_interceptions',
 'power_jumping',
 'defending_marking_awareness',
 'movement_agility',
 'defending_standing_tackle',
 'defending_sliding_tackle',
 'movement_sprint_speed',
 'movement_acceleration',
 'pace',
 'weight_kg',
 'movement_balance',
 'height_cm',
 'goalkeeping_positioning',
 'goalkeeping_handling',
 'goalkeeping_reflexes',
 'goalkeeping_diving',
 'goalkeeping_kicking',
 'overall']

In [80]:
legacy= pd.read_csv('legacy.csv', usecols=corrcolumns)

In [81]:
# filling missing values
legacy.fillna(0, inplace=True)

#making overall of 92 and 93 into 92
legacy['overall'] = legacy['overall'].apply(lambda x: 92 if x in [92, 93] else x)


In [82]:
# Shooting
shooting_attributes = ['shooting', 'power_shot_power', 'power_long_shots', 'attacking_volleys', 'attacking_finishing']
legacy['shooting_skills'] = legacy[shooting_attributes].mean(axis=1)
legacy.drop(columns=shooting_attributes, inplace=True)
missing_attributes = [attr for attr in shooting_attributes if attr not in legacy.columns]
if missing_attributes:
    print(f"Warning: The following shooting attributes are missing: {missing_attributes}")
else:
    legacy['shooting_skills'] = legacy[shooting_attributes].mean(axis=1)
    legacy.drop(columns=shooting_attributes, inplace=True)

In [83]:
#Mentality
mentality_attributes = ['mentality_aggression', 'mentality_interceptions', 'mentality_positioning',
                        'mentality_vision', 'mentality_penalties', 'mentality_composure']
imputer = SimpleImputer(strategy='mean')
legacy[mentality_attributes] = imputer.fit_transform(legacy[mentality_attributes])
legacy['mentality'] = legacy[mentality_attributes].mean(axis=1)
legacy.drop(columns=mentality_attributes, inplace=True)

In [84]:
#Technicality
skill_attributes = ['skill_long_passing', 'skill_ball_control', 'skill_curve', 'skill_fk_accuracy', 'skill_dribbling']
legacy['technical_skills'] = legacy[skill_attributes].mean(axis=1)
legacy.drop(columns=skill_attributes, inplace=True)


In [85]:
#Goalkeeping
goalkeeping_attributes = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                          'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
legacy[goalkeeping_attributes] = imputer.fit_transform(legacy[goalkeeping_attributes])
legacy['goalkeeping_ability'] = legacy[goalkeeping_attributes].mean(axis=1)
legacy.drop(columns=goalkeeping_attributes, inplace=True)


In [86]:
#converting to integer type
legacy = legacy.astype(int)

#picking target variable and its features
y = legacy['overall']
X = legacy.drop(columns=['overall'])

In [87]:
# Standardizing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)



In [88]:
# Training and Evaluating
# Random Forest Regressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print(f'Mean Absolute Error for Random Forest: {mae_rf}')

Mean Absolute Error for Random Forest: 0.6004523794789282


In [89]:
# XGBoost Regressor
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f'Mean Absolute Error for XGBoost: {mae_xgb}')

Mean Absolute Error for XGBoost: 0.6788425198860766


In [90]:
# Gradient Boosting Regressor
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
print(f'Mean Absolute Error for Gradient Boosting: {mae_gb}')

Mean Absolute Error for Gradient Boosting: 0.942441736725279


In [ ]:
rf_params = {
    'n_estimators': [50,100],
    'max_depth': [5, 10],
    'min_samples_split': [2,10]
}
grid_rf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=rf_params, scoring='neg_mean_absolute_error', cv=3)
grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_

# Hyperparameter tuning for XGBoost
xgb_params = {
    'n_estimators': [50,100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}
grid_xgb = GridSearchCV(estimator=XGBRegressor(), param_grid=xgb_params, scoring='neg_mean_absolute_error', cv=3)
grid_xgb.fit(X_train, y_train)
best_xgb = grid_xgb.best_estimator_


In [ ]:

ensemble_model = VotingRegressor(estimators=[('rf', best_rf), ('xgb', best_xgb)])
ensemble_model.fit(X_train, y_train)
ensemble_predictions = ensemble_model.predict(X_test)
mae_ensemble = mean_absolute_error(y_test, ensemble_predictions)
print(f'Mean Absolute Error for Ensemble Model: {mae_ensemble}')


In [ ]:
#saving scalar
joblib.dump(scaler, 'scaler_ensemble.pkl')

#saving ensembling model
filename = 'sports_prediction_ensemble_model.pkl'
pkl.dump(ensemble_model, open(filename, 'wb'))
